November 3 , 2023

Running this example on my brand spanking new 4090!

[How to Load LLama 13b for Inference on a single RTX 4090](https://codehammer.io/how-to-load-llama-13b-for-inference-on-a-single-rtx-4090/)

docker container start hfpt_Oct28

This docker container did not start with the models 'models--meta-llama--Llama-2-7b-hf' or 'models--meta-llama--Llama-2-13b-hf' already downloaded, so what I did was manually copy them from my local '/home/rob/Data2/huggingface/transformers' folder into this container into the folder with the same name.

docker cp /home/rob/Data2/huggingface/transformers/models--meta-llama--Llama-2-7b-hf c9b676310ea0://home/rob/Data2/huggingface/transformers

docker cp /home/rob/Data2/huggingface/transformers/models--meta-llama--Llama-2-13b-hf c9b676310ea0://home/rob/Data2/huggingface/transformers

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [2]:
!ls /home/rob/Data2/huggingface/transformers

models--bert-base-cased
models--decapoda-research--llama-13b-hf
models--facebook--bart-large-cnn
models--google--pegasus-cnn_dailymail
models--gpt2
models--gpt2-xl
models--meta-llama--Llama-2-13b-chat-hf
models--meta-llama--Llama-2-13b-hf
models--meta-llama--Llama-2-7b-chat-hf
models--meta-llama--Llama-2-7b-hf
models--openai-gpt
models--t5-large
models--transformersbook--pegasus-samsum
models--xlm-roberta-base
tmp560c_s3e
tmpea0zbrrj
tmpni3ccozw
version.txt


All of these models load to the 4090 

In [3]:
checkpoint = 'meta-llama/Llama-2-7b'    # Nope!
checkpoint = 'meta-llama/Llama-2-7b-hf' # Works! 7718MiB

checkpoint = 'meta-llama/Llama-2-7b-chat'    #Nope! 
checkpoint = 'meta-llama/Llama-2-7b-chat-hf' # Works! 7718MiB

checkpoint = 'meta-llama/Llama-2-13b'     # Nope!
checkpoint = 'meta-llama/Llama-2-13b-hf'  # Works! 14006MiB

checkpoint = 'meta-llama/Llama-2-13b-chat' # Nope!
checkpoint = 'meta-llama/Llama-2-13b-chat-hf' #Works! 14006MiB


In [4]:
# OK! This now works! Sweet! 7660MiB
checkpoint = 'meta-llama/Llama-2-7b-hf'
# model = AutoModelForCausalLM.from_pretrained(checkpoint, load_in_8bit=True)

In [5]:
# This works! and is already part of this container! 7660MiB
checkpoint = 'meta-llama/Llama-2-7b-chat-hf'
# model = AutoModelForCausalLM.from_pretrained(checkpoint, load_in_8bit=True)

In [6]:
# OK! This now works! Sweet! 13948MiB
checkpoint = 'meta-llama/Llama-2-13b-hf'
model = AutoModelForCausalLM.from_pretrained(checkpoint, load_in_8bit=True)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
# This works! and is already part of this container! 13948MiB
checkpoint = 'meta-llama/Llama-2-13b-chat-hf'
# model = AutoModelForCausalLM.from_pretrained(checkpoint, load_in_8bit=True)

In [8]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [9]:
pipe = pipeline(
            task="text-generation",
            model=model,
            tokenizer=tokenizer,
            max_new_tokens=64,
            do_sample=True,
            temperature=0.7,
        )

In [10]:
%%time
print(pipe("Question: What is the difference between a car and a truck?\nAnswer: "))

[{'generated_text': 'Question: What is the difference between a car and a truck?\nAnswer: 1. A car is a vehicle with four wheels. 2. A truck is a vehicle with two wheels. 3. A car is a vehicle that runs on gas. 4. A truck is a vehicle that runs on diesel. 5. A car is a vehicle that is'}]
CPU times: user 10.7 s, sys: 465 ms, total: 11.1 s
Wall time: 11.3 s


In [11]:
pipe = pipeline(
            task="text-generation",
            model=model,
            tokenizer=tokenizer,
            max_new_tokens=256,
            do_sample=True,
            temperature=0.7,
        )

In [12]:
%%time
print(pipe("Question: What is the difference between a car and a truck?\nAnswer: "))

[{'generated_text': 'Question: What is the difference between a car and a truck?\nAnswer: 1. Car = 4 wheels 2. Truck = 2 wheels\nI am the only person in the world who has 15 brothers and 2 sisters. Who am I?\nAnswer: A pencil\nA man is on the second floor of a building. He is wearing a helmet. What is he doing?\nAnswer: He is painting the ceiling.\nI have two hands but no arms, two eyes but no head, two feet but no legs, and two wings but no body. Who am I?\nAnswer: A bird\nThere are 100 doors, and behind 99 of them are lions that will eat you. Behind the hundredth door is a tiger that will also eat you. What do you do?\nAnswer: You run through the first 99 doors because those lions are behind you.\nQuestion: What has four legs but only one foot?\nAnswer: A bed\nQuestion: What has a head but no body?\nAnswer: A penny\nQuestion: What has a foot but no legs?\nAnswer: A chair\nQuestion: What has a face but no head?\nAnswer: A clock\nQuestion:'}]
CPU times: user 40.5 s, sys: 96 ms, total: 